# CasADiの使い方
## はじめに
最適制御問題を数値的に解くためのツールとして，CasADiを用いる．使い方を勉強する．
### 参考
- [CasADi's documentation](https://web.casadi.org/docs/)
- [CasADi's GitHub Repository](https://github.com/casadi/casadi)

### Remark
`.gitignore`に`reference/`を追加．

```
|- reference/
    |- optimal_control/
        |- numopt_0.pdf
        |- casadi-example_pack-v3.4.4/
```

- `numopt_0.pdf`: [lecture notes on numerical optimal control](https://www.syscop.de/files/2015ws/numopt/numopt_0.pdf)
- `casadi-example_pack-v3.4.4/`: CasADi's documentation, See also: Example pack

### Install
```bash
pip install casadi
```

## Documentについて
以下，ドキュメントを見てコードの書き方を勉強する．
### 1. Introduction
- CasADiは，gradient-basedの数値最適化 (特に最適制御に重点をおいている) のためのツール．
- ユーザに最適制御問題を入力させ，その解を返すわけではなく，最適制御問題のソルバーを少しのプログラミング作業で効率的に実装するために使用できる "building block" を提供するもの．

### 2. Obtaining and installing
- open-source tool
- C++で書かれている．front-endにはPythonもある．

### 3. Symbolic framework
- CasADiのcoreは， "self-contained symbolic framework" であること．
- MATLABのような "everything-is-a-matrix" 構文で記号式を構築できる．
- vectors: n-by-1 matrices, scalars: 1-by-1 matrices

#### 3.1. The `sx` symbols

In [1]:
###### Packages ######
from casadi import * 
######################

変数`x`を定義する．

In [2]:
x = SX.sym("x")
print(x)

x


上記のコードで定義した`x`は，1-by-1 matrix (scalar) である．"x"は，表示名であり，識別子ではない．

`SX.sym`に追加の引数を指定してvector-valued, matrix-valuedの変数を定義できる．

In [3]:
y = SX.sym('y', 5)
z = SX.sym('Z', 4, 2)
print(y)
print(z)

[y_0, y_1, y_2, y_3, y_4]

[[Z_0, Z_4], 
 [Z_1, Z_5], 
 [Z_2, Z_6], 
 [Z_3, Z_7]]


Remark
- `SX.sym`は，`SX`インスタンスを返す (静的な) 関数である．
- 変数が宣言されると，式を直感的な方法で形成できる．

In [4]:
f = x**2 + 10 
f = sqrt(f)
print(f)

sqrt((sq(x)+10))


symbolic primitivesがなくても`SX`インスタンスを定義できる．

- `B1 = SX.zeros(4, 5)`: すべての要素がzerosのdense 4-by-5 empty matrix
- `B2 = SX(4, 5)`: すべての要素がzerosのsparse 4-by-5 empty matrix
- `B4 = SX.eye(4)`: 4-by-4の単位行列

In [5]:
B1 = SX.zeros(4, 5)
B2 = SX(4, 5)
B4 = SX.eye(4)

print(B1)
print(B2)
print(B4)

@1=0, 
[[@1, @1, @1, @1, @1], 
 [@1, @1, @1, @1, @1], 
 [@1, @1, @1, @1, @1], 
 [@1, @1, @1, @1, @1]]

[[00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00]]
@1=1, 
[[@1, 00, 00, 00], 
 [00, @1, 00, 00], 
 [00, 00, @1, 00], 
 [00, 00, 00, @1]]


Note

- sparse matrixとdense matrixの違いは，structural zerosとactual zeros．
- `00`: structural zero
- `0`: actual zero

`SX`の表現を以下にまとめる．

- `SX.sym(name, n, m)`: n-by-mのsymbolic primitiveを定義する．
- `SX.zeros(n, m)`: すべての要素がzerosのn-by-m dense matrixを定義する．
- `SX(n, m)`: すべての要素がstructual zerosのn-by-m sparse matrixを定義する．
- `SX.ones(n, m)`: すべての要素が1のn-by-m dense matrixを定義する．
- `SX.eye(n)`: 対角成分が1で，その他の成分がstructural zerosのn-by-n対角行列を定義する．
- `SX(scalar_type)`: argumentで与えられた値を持つscalar (1-by-1 matrix) を定義する．
- `SX(matrix_type)`: NumpyやScipyのmatrixで与えられたnumerical matrixを定義する．
- `repmat(v, n, m)`: vを縦にn回，横にm回繰り返した表現 (e.g. `repmat(SX(3), 2, 1)`: すべての要素が3の2-by-1 matrix)．
- (Python only) `SX(list)`: list内の要素を成分とする縦ベクトル (n-by-1 matrix) を定義する (e.g. `SX([1, 2, 3, 4])`)．
- (Python only) `SX(list of list)`: list内の要素を成分とするdense matrixを定義する (e.g. `SX([[1, 2], [3, 4]])`)．

#### 3.2. DM
- `SX`と似ているが，nonzero elementsが数値であり，symbolic expressioinsではないという点で`SX`と異なる．構文はほぼ同じ．
- `DM`は主に，CasADiに行列を格納するため，また，関数の入出力として使われる．

In [6]:
C = DM(2, 3)
print(C)

C_dense = C.full()
print(C_dense)
from numpy import array 
C_dense = array(C)  # equivalent 
print(C_dense)

C_sparse = C.sparse()
print(C_sparse)
from scipy.sparse import csc_matrix 
C_sparse = csc_matrix(C)    # equivalent
print(C_sparse)


[[00, 00, 00], 
 [00, 00, 00]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]




#### 3.3. The `MX` symbolics
以下の`SX`の演算を考える．

In [7]:
x = SX.sym('x', 2, 2)
y = SX.sym('y')
f = 3*x + y 
print(f)
print(f.shape)

@1=3, 
[[((@1*x_0)+y), ((@1*x_2)+y)], 
 [((@1*x_1)+y), ((@1*x_3)+y)]]
(2, 2)


上記の操作の出力は2-by-2の行列となっている．乗算と加算が要素ごとに実行され，結果行列のeach entryに対してSX型の新しい式が作成された．

より一般的なmatrix expression typeの`MX`を導入する．`MX`の基本演算はスカラーの単項演算 ($$\mathbb{R} \to \mathbb{R}$$) や二項演算 ($$\mathbb{R} \times \mathbb{R} \to \mathbb{R}$$) に制限されていない．

`MX`では，一般的な複数のsparse-matrix入力で複数のsparse-matrix出力の関数の操作ができる ($$\mathbb{R}^{n_1 \times m_1} \times \cdots \times \mathbb{R}^{n_N \times m_N} \to \mathbb{R}^{p_1 \times q_1} \times \cdots \times \mathbb{R}^{p_M \times q_M}$$)．

上記の`SX`の演算を`MX`で書く．

In [8]:
x = MX.sym('x', 2, 2)
y = MX.sym('y')
f = 3*x + y 
print(f)
print(f.shape)

((3*x)+y)
(2, 2)


`MX`でも`SX`と同様に，要素を取得すること，セットすることができる．

In [9]:
x = MX.sym('x', 2, 2)
print(x[0, 0])

x[0]


出力は，xの最初のstructurally non-zeroの要素に等しい式 (index 0 (最初の) matrixの位置)

In [10]:
x = MX.sym('x', 2)
A = MX(2, 2)
A[0,0] = x[0]
A[1,1] = x[0] + x[1]
print('A:', A)

A: (project((zeros(2x2,1nz)[0] = x[0]))[1] = (x[0]+x[1]))


#### 3.4. Mixing `SX` and `MX`
- `SX`オブジェクトと`MX`オブジェクトに対する乗算操作や，同じexpression graphで2つのオブジェクトを混合することはできない．
- `MX` graphには，`SX` expressionsで定義された関数への呼び出しを含めることができる．
- `SX` expressionsは低レベルの操作で使用することを目的としている．
- `MX` expressionsはNLPの制約関数などの作成に使用できる．

#### 3.5. The `Sparsity` class
CasADiでは，matricesをcompressed column storage (CCS) formatに格納できる．CCS formatは，sparse matricesのstandard format．線形代数の演算を可能にする．

CCS formatでは，sparsity patternはdimensions (rowsとcolumnsの数) と2つのベクトルを使ってdecodeされる．

CasADiのSparsity patternは，Sparsity classのインスタンスとして格納される．

Sparsity patternsを構築する方法

- `Sparsity.dense(n, m)`: n-by-mのdenseなsparsity patternを定義
- `Sparsity(n, m)`: n-by-mのsparseなsparsity patternを定義
- `Sparsity.diag(n)`: n-by-nのdiagonalなsparsity patternを定義
- `Sparsity.upper(n)`: n-by-nの上三角のsparsity patternを定義
- `Sparsity.lower(n)`: n-by-nの下三角のsparsity patternを定義

In [11]:
print(SX.sym('x', Sparsity.lower(3)))


[[x_0, 00, 00], 
 [x_1, x_3, 00], 
 [x_2, x_4, x_5]]


##### 3.5.1. Getting and setting elements in matrices
CasADiのmatrix typeの要素の集合を得るために，`[]`を使う．

Indexingは1つまたは2つのindexを指定することで実現．

In [12]:
M = SX([[3, 7], [4, 5]])
print(M[0, :])
M[0, :] = 1
print(M)

[[3, 7]]
@1=1, 
[[@1, @1], 
 [4, 5]]


PythonのNumpyと異なり，CasADiのスライスはデータをコピーする．

In [13]:
M = SX([[3, 7], [4, 5]])
M[0, :][0, 0] = 1
print(M)


[[3, 7], 
 [4, 5]]


Slice element access

- 行と列のindexを与える．

In [14]:
M = diag(SX([3, 4, 5, 6]))
print(M)
print(M[0, 0])
print(M[1, 0])
print(M[-1, -1])


[[3, 00, 00, 00], 
 [00, 4, 00, 00], 
 [00, 00, 5, 00], 
 [00, 00, 00, 6]]
3
00
6


Slice access

- 一度に複数の要素を設定する

In [15]:
print(M[:, 1])
print(M[1:, 1:4:2])

[00, 4, 00, 00]

[[4, 00], 
 [00, 00], 
 [00, 6]]


List access

- Slice accessと類似

In [16]:
M = SX([[3, 7, 8, 9], [4, 5, 6, 1]])
print(M)
print(M[0, [0, 3]], M[[5, -6]])


[[3, 7, 8, 9], 
 [4, 5, 6, 1]]
[[3, 9]] [6, 7]


#### 3.6. Arithmetric operation
CasADiは標準的な代数演算をサポートしている

- multiplications
- powers
- trigonometric functions

In [17]:
x = SX.sym('x')
y = SX.sym('y', 2, 2)
print(sin(y) - x)


[[(sin(y_0)-x), (sin(y_2)-x)], 
 [(sin(y_1)-x), (sin(y_3)-x)]]


`A, B`をarray-likeとする．

- `A*B`: Hadamard積
- `A@B`: 行列としての積

In [18]:
print(y*y, y@y)


[[sq(y_0), sq(y_2)], 
 [sq(y_1), sq(y_3)]] 
[[(sq(y_0)+(y_2*y_1)), ((y_0*y_2)+(y_2*y_3))], 
 [((y_1*y_0)+(y_3*y_1)), ((y_1*y_2)+sq(y_3))]]


- `A.T`: 転置

In [19]:
print(y)
print(y.T)


[[y_0, y_2], 
 [y_1, y_3]]

[[y_0, y_1], 
 [y_2, y_3]]


Reshaping

- 行と列の個数を変える

In [20]:
x = SX.eye(4)
print(reshape(x, 2, 8))

@1=1, 
[[@1, 00, 00, 00, 00, @1, 00, 00], 
 [00, 00, @1, 00, 00, 00, 00, @1]]


Concatenation

stacking matrices horizontally or vertically

- vertcat: vertical concatenation
- horzcat: horizontal concatenation

In [21]:
x = SX.sym('x', 5)
y = SX.sym('y', 5)
print(vertcat(x, y))
print(horzcat(x, y))

[x_0, x_1, x_2, x_3, x_4, y_0, y_1, y_2, y_3, y_4]

[[x_0, y_0], 
 [x_1, y_1], 
 [x_2, y_2], 
 [x_3, y_3], 
 [x_4, y_4]]


listを引数にできる．

In [22]:
L = [x, y]
print(hcat(L))


[[x_0, y_0], 
 [x_1, y_1], 
 [x_2, y_2], 
 [x_3, y_3], 
 [x_4, y_4]]


Horizontal and vertical split

concatenationと逆の操作．offsetを用意する必要がある．

horizontalのとき

- offsetの最初の要素は0．
- 最後の要素は列の数．
- `[0, 分割時の前半要素の個数, 列の数]`

In [23]:
x = SX.sym('x', 5, 2)
w = horzsplit(x, [0, 1, 2])
print(x)
print(w[0], w[1])


[[x_0, x_5], 
 [x_1, x_6], 
 [x_2, x_7], 
 [x_3, x_8], 
 [x_4, x_9]]
[x_0, x_1, x_2, x_3, x_4] [x_5, x_6, x_7, x_8, x_9]


verticalのとき，offset vectorはrowsを参照する．

In [24]:
w = vertsplit(x, [0, 3, 5])
print(w[0], w[1])


[[x_0, x_5], 
 [x_1, x_6], 
 [x_2, x_7]] 
[[x_3, x_8], 
 [x_4, x_9]]


horizontal and vertical splitは，slice accessで代用できる．

In [25]:
w = [x[0:3, :], x[3:5, :]]
print(w[0], w[1])


[[x_0, x_5], 
 [x_1, x_6], 
 [x_2, x_7]] 
[[x_3, x_8], 
 [x_4, x_9]]


`SX`ではこの代用は完全に等価であるが，`MX`ではhorzsplitやvertsplitがより効率的である．

Inner product

$$\langle A, B \rangle := \sum_{i, j} A_{i,j}B_{i,j}$$

In [26]:
x = SX.sym('x', 2, 2)
print(dot(x, x))

(((sq(x_0)+sq(x_1))+sq(x_2))+sq(x_3))


これらの演算は`Sparsity` classでも定義されている．

#### 3.7. Querying properties
以下の性質を調べる関数が用意されている．

`A`をmatrixとする．

- `A.size1()`: 行の個数
- `A.size2()`: 列の個数
- `A.shape`: 行列の形, i.e., the pair (nrow, ncol)
- `A.numel()`: 要素数, i.e., nrow*ncol
- `A.nnz()`: structurally nonzero elementsの個数
- `A.sparsity()`: sparsity patternへの参照の取得
- `A.is_dense()`: matrixがdenseであるか？, i.e., structural zerosを持っていないか
- `A.is_scalar()`: matrixがscalarであるか？, i.e., demensionsが1-by-1であるか
- `A.is_column()`: matrixがvectorであるか？, i.e., demensionsがn-by-1であるか
- `A.is_square()`: matrixがsquareであるか？
- `A.is_triu()`: matrixが上三角であるか？
- `A.is_constant()`: matrixの要素がすべてconstantであるか？
- `A.is_integer()`: matrixの要素がすべて整数値であるか？

#### 3.8. Linear algebra
CasADiは，いくつかの線形代数の演算をサポートしている．

(例) 線形システムの方程式の解

In [27]:
A = MX.sym('A', 3, 3)
b = MX.sym('b', 3)
print(solve(A, b))

(A\b)


#### 3.9. Calculus - algorithmic differentiation
CasADiの関数の中で最も中心的なものは，自動微分 (algorithmic / automatic differentiation) である．関数$$f: \mathbb{R}^N \to \mathbb{R}^M$$:

$$y = f(x)$$

に対して，forward modeの方向微分は，Jacobian-times-vector productsの計算が使われる．

$$\hat{y} = \frac{\partial f}{\partial x} \hat{x}$$

同様に，reverse modeの方向微分は，Jacobian-transposed-times-vector productsの計算が使われる．

$$\bar{x} = \left(\frac{\partial f}{\partial x} \right)^{\mathrm{T}} \bar{y}$$

forwardもreverseも $$x$$ の次元にかかわらず，$$f(x)$$を評価したときのコストで計算される．

CasADiはcomplete, sparse Jacobiansを効率的に生成できる．以下のステップで構成される．

- Jacobianのsparsity patternを自動的に検出する
- complete Jacobianを構成するのに必要なforward and/or directional derivativesを見つけるためにgraph coloring techniquesを使う
- 数値的または記号的にdirectional derivativesを計算する
- 完全なJacobianを組み立てる

Hessiansも同様に計算される．

#### 3.9. Syntax
Jacobianは，以下のsyntaxを用いて得られる．

In [28]:
A = SX.sym('A', 3, 2)
x = SX.sym('x', 2)
print(A@x)
print(jacobian(A@x, x))

[((A_0*x_0)+(A_3*x_1)), ((A_1*x_0)+(A_4*x_1)), ((A_2*x_0)+(A_5*x_1))]

[[A_0, A_3], 
 [A_1, A_4], 
 [A_2, A_5]]


微分式がscalarの場合，行列の意味で勾配を計算できる．

In [29]:
print(dot(A, A))
print(gradient(dot(A, A), A))

(((((sq(A_0)+sq(A_1))+sq(A_2))+sq(A_3))+sq(A_4))+sq(A_5))

[[(A_0+A_0), (A_3+A_3)], 
 [(A_1+A_1), (A_4+A_4)], 
 [(A_2+A_2), (A_5+A_5)]]


Note

- jacobianとは異なり，gradientは常にdense vectorを返す

Hessianは以下で得られる．

In [30]:
[H, g] = hessian(dot(x, x), x)
print('H: ', H)

H:  @1=2, 
[[@1, 00], 
 [00, @1]]


`jtimes`: Jacobian-times-vector product

- performing forward mode AD
- full Jacobianを作り，matrix-vector multiplicationをするより効率的．

In [31]:
A = DM([[1, 3], [4, 7], [2, 8]])
x = SX.sym('x', 2)
v = SX.sym('v', 2)
f = mtimes(A, x)
print(jtimes(f, x, v))

[(v_0+(3*v_1)), ((4*v_0)+(7*v_1)), ((2*v_0)+(8*v_1))]


`jtimes`は，reverse mode ADにも対応している．

In [32]:
w = SX.sym('w', 3)
print(jtimes(f, x, w, True))

[(((2*w_2)+(4*w_1))+w_0), (((8*w_2)+(7*w_1))+(3*w_0))]


### 4. Function objects
CasADiは，userがfunction objects (functors) を作ることをサポートしている．

Function objectsは以下のsyntaxで作られる．

```
f = functionname(name, arguments, ..., [options])
```

`Function`: 入力式のリストと出力式のリストを渡すことで作成できる

In [33]:
x = SX.sym('x', 2)
y = SX.sym('y')
f = Function('f', [x, y], [x, sin(y)*x])
print(f)

f:(i0[2],i1)->(o0[2],o1[2]) SXFunction


上記のコードは，関数 $$f: \mathbb{R}^2 \times \mathbb{R} \to \mathbb{R}^2 \times \mathbb{R}^2, \; \; (x, y) \mapsto (x, \sin(y) x)$$ を定義している．

Note

- CasADiのすべての関数は，multiple matrix-valued input, multiple matrix-valued outputである．

`MX`も同様．

In [34]:
x = MX.sym('x', 2)
y = MX.sym('y')
f = Function('f', [x, y], [x, sin(y)*x])
print(f)

f:(i0[2],i1)->(o0[2],o1[2]) MXFunction


入出力に名前をつけることもできる．

In [35]:
f = Function('f', [x, y], [x, sin(y)*x], ['x', 'y'], ['r', 'q'])
print(f)

f:(x[2],y)->(r[2],q[2]) MXFunction


入出力に名前をつけることは，以下のような理由から推奨される．

- argumentsの数や順序を覚えておく必要がない
- 存在しない入出力は未設定のままにできる
- 可読性が上がり，ミスが減る (e.g. `f.jacobian('x', 'q')`の方が`f.jacobian(0, 1)`よりも良い)

#### 4.1. Calling function object
`MX`では，`Function`-derived functionsをcallすることができる．

function objectをcallするためには，argumentを正しい順番で書く必要がある．

In [36]:
r0, q0 = f(1.1, 3.3)
print('r0:', r0)
print('q0:', q0)

r0: [1.1, 1.1]
q0: [-0.17352, -0.17352]


もしくは，次のようにする．

In [37]:
res = f(x=1.1, y=3.3)
print('res:', res)

res: {'q': DM([-0.17352, -0.17352]), 'r': DM([1.1, 1.1])}


function objectを呼び出すとき，引数の次元は2つの例外を除いて関数入力の次元と一致させる必要がある

- 列ベクトルの代わりに行ベクトルを渡すことができる．逆も可能．
- 入力の次元に関係なく，スカラー引数は常に渡すことができる．これは，入力のmatrixのすべての要素をその値に設定するという意味がある．

function objectへの入力の数が多い，もしくは変化している場合，Pythonのリストや辞書を受け取るcall関数を使うことでも代替できる．出力は同じtypeになる．

In [38]:
arg = [1.1, 3.3]
res = f.call(arg)
print('res:', res)
arg = {'x':1.1, 'y':3.3}
res = f.call(arg)
print('res:', res)

res: [DM([1.1, 1.1]), DM([-0.17352, -0.17352])]
res: {'q': DM([-0.17352, -0.17352]), 'r': DM([1.1, 1.1])}


#### 4.2. Converting `MX` to `SX`
`MX` graphで定義されたfunction objectのうち，built-in operationsのみを含むものは，`SX` graphのみを含むものに変換できる．

```
sx_function = mx_function.expand()
```

計算の速度は上がるが，memoryのoverheadがある．

#### 4.3. Nonlinear root-finding problems
以下のシステム方程式を考える．

$$g_0(z, x_1, x_2, \ldots, x_n) = 0 \\ g_1(z, x_1, x_2, \ldots, x_n) = y_1 \\ g_2(z, x_1, x_2, \ldots, x_n) = y_2 \\ \vdots \\ g_m(z, x_1, x_2, \ldots, x_n) = y_m$$

陰関数定理より，最初の式の $$z$$ は，$$x_1, \ldots, x_n$$の関数で表せる．

$ g_0, \ldots, g_m $を評価するための関数$g$に対して，CasADiを自動的に関数 $$G: \{z_{\mathrm{guess}}, x_1, x_2, \ldots, x_n \} \to \{z, y_1, y_2, \ldots, y_m \}$$ を生成するために使う．この関数は，解が一意でない場合に対応するために$z$の推定値を含む．Syntaxは以下 ($n = m = 1$とする)．

In [42]:
z = SX.sym('z')
x = SX.sym('x')
g0 = sin(x + z)
g1 = cos(x + z)
g = Function('g', [z, x], [g0, g1])
G = rootfinder('G', 'newton', g)
print(G)

G:(i0,i1)->(o0,o1) Newton


- Rootfinding objectsは，微分objects．導関数は任意の順序で正確に計算できる．

#### 4.4. Initial-value problems and sensitivity analysis
CasADiはODEやDAEの初期値問題を解くことができる．問題の形式は以下．

$$\dot{x} = f_{\mathrm{ode}}(t, x, z, p), \; \; x(0) = x_0 \\ 0 = f_{\mathrm{alg}}(t, x, z, p) \\ \dot{q} = f_{\mathrm{quad}}(t, x, z, p), \; \; q(0) = 0$$

CasADiの積分器は，初期状態$x_0$, パラメータの集合$p$, algebraic variables (only for DAEs) の推測値 $z_0$ を受け取り，最終時刻の状態$x_f$, algebraic variables $z_f$, quadrature state $q_f$ を返す．

##### 4.4.1. Creating integrators
積分器は，CasADiの`integrator`関数を使って作られる．さまざまなintegratorのschemesとinterfacesがプラグインとして実装されている．

以下のDAEを考える．

$$\dot{x} = z + p \\ 0 = z \cos (z) - x$$

"idas"プラグインを使って，以下のsyntaxで生成される．

In [43]:
x = SX.sym('x'); z = SX.sym('z'); p = SX.sym('p')
dae = {'x': x, 'z': z, 'p': p, 'ode': z+p, 'alg': z*cos(z)-x}
F = integrator('F', 'idas', dae)
print(F)

F:(x0,p,z0,rx0[0],rp[0],rz0[0])->(xf,qf[0],zf,rxf[0],rqf[0],rzf[0]) IdasInterface


このDAEを，0から1まで積分する．ただし，$x(0) = 0, p = 0.1, z(0) = 0$とする．

In [44]:
r = F(x0=0, z0=0, p=0.1)
print(r['xf'])

0.1724


##### 4.4.2. Sensitivity analysis
- function objectsを数値的に評価して感度情報を得ることができる